# Data File Preview - ASC and TDMS Files

This notebook lets you preview how your ASC or TDMS files would look when converted to Parquet format.
No files are written - just viewing!

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## Helper Functions

In [ ]:
def load_and_process_asc_file(file_name):
    """Load and process ASC file."""
    # Read the ASC file
    df = pd.read_csv(file_name, delim_whitespace=True)
    
    # Fill NaN values with 0.0
    df = df.fillna(0.0)
    
    return df

In [ ]:
def load_and_process_tdms_file(file_name):
    """Load TDMS file."""
    from nptdms import TdmsFile

    with TdmsFile.open(file_name) as tdms_file:
        # Get all groups in the file
        groups = tdms_file.groups()

        # Create a dictionary to store data from all groups
        data_dict = {}

        for group in groups:
            for channel in group.channels():
                channel_name = f"{group.name}/{channel.name}"
                data = channel[:]
                data_dict[channel_name] = data

        # Find the maximum length of data
        max_length = max(len(data) for data in data_dict.values())

        # Pad shorter arrays with NaN
        for key in data_dict:
            if len(data_dict[key]) < max_length:
                pad_length = max_length - len(data_dict[key])
                data_dict[key] = np.pad(
                    data_dict[key],
                    (0, pad_length),
                    'constant',
                    constant_values=np.nan
                )

        # Create DataFrame
        df = pd.DataFrame(data_dict)
        df = df.fillna(0.0)
        return df

## Step 1: Enter Your File Path

Change the path below to point to your ASC or TDMS file:

In [ ]:
# 👇 CHANGE THIS to your file path
input_file = "source_folder/your_file.asc"  # or .tdms

# Or use this format if you want to specify a complete path:
# input_file = "/full/path/to/your/file.asc"
# input_file = "/full/path/to/your/file.tdms"

## Step 2: Load and Preview the Data

In [ ]:
# Detect file type and load accordingly
file_path = Path(input_file)
file_extension = file_path.suffix.lower()

print(f"Loading file: {input_file}")
print(f"File type: {file_extension}")
print()

if file_extension in ['.asc', '.sc']:
    df = load_and_process_asc_file(input_file)
    print("✓ Loaded ASC file")
elif file_extension == '.tdms':
    df = load_and_process_tdms_file(input_file)
    print("✓ Loaded TDMS file")
else:
    raise ValueError(f"Unsupported file type: {file_extension}. Use .asc, .sc, or .tdms")

print(f"✓ Shape: {df.shape[0]} rows × {df.shape[1]} columns")

## Step 3: View Basic Information

In [ ]:
# Show column names
print("Column names:")
print(df.columns.tolist())
print(f"\nTotal columns: {len(df.columns)}")

In [ ]:
# Show data types
print("Data types:")
print(df.dtypes)

## Step 4: Preview the Data (First Few Rows)

In [ ]:
# Show first 10 rows
df.head(10)

## Step 5: Check for Potential Issues

In [ ]:
# Check for columns with all zeros
all_zero_cols = []
for col in df.columns:
    if (df[col].fillna(0) == 0.0).all():
        all_zero_cols.append(col)

if all_zero_cols:
    print(f"⚠️ Found {len(all_zero_cols)} columns with all zeros:")
    print(all_zero_cols)
else:
    print("✓ No all-zero columns found")

In [ ]:
# Check for unnamed or invalid columns
invalid_cols = []
for col in df.columns:
    col_str = str(col).strip()
    if (not col_str or col_str == '' or 
        col_str.startswith('Unnamed:') or 
        col_str.lower() == 'nan' or pd.isna(col)):
        invalid_cols.append(col)

if invalid_cols:
    print(f"⚠️ Found {len(invalid_cols)} columns with invalid names:")
    print(invalid_cols)
else:
    print("✓ All columns have valid names")

## Step 6: View Basic Statistics

In [ ]:
# Show summary statistics
df.describe()

## Optional: Check for Missing Values (Before fillna was applied)

In [ ]:
# Note: Since both functions apply fillna(0.0), there should be no missing values
# But we can check anyway
missing = df.isnull().sum()
missing = missing[missing > 0]

if len(missing) > 0:
    print("Columns with missing values:")
    print(missing)
else:
    print("✓ No missing values (all NaNs were filled with 0.0)")

## Optional: View Last Few Rows

In [ ]:
# Show last 10 rows
df.tail(10)

## Optional: View Sample of Data

In [ ]:
# Show random sample of 10 rows
df.sample(min(10, len(df)))

## Optional: Check Memory Usage

In [ ]:
# Show memory usage
memory_usage = df.memory_usage(deep=True).sum() / 1024**2  # Convert to MB
print(f"DataFrame memory usage: {memory_usage:.2f} MB")

## Filter Columns - Keep Only Specific Columns

This section shows you how to keep only the columns you need.

In [ ]:
# Method 1: Select specific columns by name (RECOMMENDED)
# List the exact columns you want to keep
columns_to_keep = [
    'Messzeit[s]',
    'Pressure [bar]',
    'Flow [L/min]',
    'Leak [L/min]',
    'Torque [Nm]'
]

# Create filtered dataframe
df_filtered = df[columns_to_keep]

print(f"Original DataFrame: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Filtered DataFrame: {df_filtered.shape[0]} rows × {df_filtered.shape[1]} columns")
print(f"\nRemoved {df.shape[1] - df_filtered.shape[1]} columns")
print(f"\nKept columns: {df_filtered.columns.tolist()}")

In [ ]:
# Preview the filtered data
print("First 10 rows of filtered data:")
df_filtered.head(10)

In [ ]:
# Method 2: Drop unwanted columns (Alternative approach)
# List the columns you want to REMOVE
columns_to_remove = [
    'Speed [rpm]',
    'LS [bar]',
    'Housing [bar]',
    'TempSaug [°C]',
    'TempLeak [°C]'
]

# Create filtered dataframe by dropping columns
df_filtered_alt = df.drop(columns=columns_to_remove, errors='ignore')

print(f"Alternative method result: {df_filtered_alt.shape[0]} rows × {df_filtered_alt.shape[1]} columns")
print(f"Columns: {df_filtered_alt.columns.tolist()}")

In [ ]:
# Method 3: Filter by pattern (if you have many similar columns)
# Keep columns that contain certain keywords
keywords = ['Messzeit', 'Pressure', 'Flow', 'Leak', 'Torque']

# Find columns matching any keyword
matching_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]

df_filtered_pattern = df[matching_columns]

print(f"Pattern-based filtering result: {df_filtered_pattern.shape[0]} rows × {df_filtered_pattern.shape[1]} columns")
print(f"Columns: {df_filtered_pattern.columns.tolist()}")

In [ ]:
# Compare statistics before and after filtering
print("Statistics for filtered data:")
df_filtered.describe()

## Save Filtered Data (Optional)

If you want to save the filtered data to a new file:

In [ ]:
# Uncomment and modify the path to save filtered data

# Save as Parquet (recommended for large files)
# output_path = "filtered_data.parquet"
# df_filtered.to_parquet(output_path, index=False)
# print(f"✓ Saved filtered data to {output_path}")

# Or save as CSV
# output_path = "filtered_data.csv"
# df_filtered.to_csv(output_path, index=False)
# print(f"✓ Saved filtered data to {output_path}")

## Helper: Find Columns by Partial Name

If you're not sure of the exact column names, use this to search:

In [ ]:
# Search for columns containing a specific word
search_term = "Temp"  # Change this to search for different terms

matching = [col for col in df.columns if search_term.lower() in col.lower()]

if matching:
    print(f"Columns containing '{search_term}':")
    for col in matching:
        print(f"  - {col}")
else:
    print(f"No columns found containing '{search_term}'")